In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
dataurl = 'https://docs.google.com/spreadsheets/d/1EmeVCJzCiMISggPuS5clv-gz4mznq24NMyFV8aXkbK4/edit?gid=0#gid=0'
dataurl = dataurl.replace('/edit?gid=', '/export?format=csv&gid=')
dataset = pd.read_csv(dataurl)
dataset = dataset.fillna('')
refDataset = dataset
dataset

,My Gender,My age,My city of residence,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else)","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc)",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist)",Name of the institute I graduated from last.,Educational degree,Anything else that you may like to mention to the community members (any specifics which may be important to you),Barring my personal details (my name and contact number) shared details can be viewed by andwemet community members.,My religious inclination,Video Introduction
0,Male,32,Mumbai,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...","Separated, filed for a divorce",No,Between 40-50 Lakhs,Finance Professional,Certified financial analyst CFA,Masters,Health concious due to health conditions in fa...,I give consent,,
1,Male,36,New Delhi,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,Institute of chartered accountants of India,Masters,I like playing sports,I give consent,,
2,Male,32,Bangalore,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),"RRCE, Bangalore",Masters,No,I give consent,,
3,Female,33,Gurugram,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,PGI Rohtak,Phd and equivalent,I take time to open up and quite curious,I give consent,,
4,Female,32,Mumbai,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,XIC,PG Diploma,I believe honest communication and mutual resp...,I give consent,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168,Male,29,Gurgaon,"A good family, have some good supportive frien...",Life is pretty good with ups and downs but the...,Never married,No,Under 12 Lakhs,I'm a Food Technologist working in a reputed MNC,CSK Himachal Pradesh Agriculture University,Undergraduate,Just be kind and spread smiles as much as you ...,I give consent,Hindu,
169,Male,44,Gurgaon,Have this life,Lonely,Was married earlier,No,Over 90 Lakhs,Entrepreneur,MiT pune,Masters,Not now,I give consent,Hindu,
170,Male,34,Gurugram,To cultivate so much self awareness to be your...,My growing up years were a bit challenging but...,Never married,No,Between 35-40 Lakhs,"Environment, Health & Safety Specialist","Faculty of Engineering , DEI",Masters,I am a high level of empath with kindness and ...,I give consent,Spiritual,
171,Female,36,Gurugram,Able to live life on my own terms,A mixed bag pf good times and bad. Military ki...,Never married,No,Between 50-70 Lakhs,Corporate and commercial lawyer,"University of York, UK",Masters,Respect and empathy are 2 core values that are...,I give consent,Hindu,


In [3]:
dataset.index = np.arange(2, len(dataset) + 2)
dataset

,My Gender,My age,My city of residence,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else)","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc)",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist)",Name of the institute I graduated from last.,Educational degree,Anything else that you may like to mention to the community members (any specifics which may be important to you),Barring my personal details (my name and contact number) shared details can be viewed by andwemet community members.,My religious inclination,Video Introduction
2,Male,32,Mumbai,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...","Separated, filed for a divorce",No,Between 40-50 Lakhs,Finance Professional,Certified financial analyst CFA,Masters,Health concious due to health conditions in fa...,I give consent,,
3,Male,36,New Delhi,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,Institute of chartered accountants of India,Masters,I like playing sports,I give consent,,
4,Male,32,Bangalore,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),"RRCE, Bangalore",Masters,No,I give consent,,
5,Female,33,Gurugram,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,PGI Rohtak,Phd and equivalent,I take time to open up and quite curious,I give consent,,
6,Female,32,Mumbai,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,XIC,PG Diploma,I believe honest communication and mutual resp...,I give consent,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,Male,29,Gurgaon,"A good family, have some good supportive frien...",Life is pretty good with ups and downs but the...,Never married,No,Under 12 Lakhs,I'm a Food Technologist working in a reputed MNC,CSK Himachal Pradesh Agriculture University,Undergraduate,Just be kind and spread smiles as much as you ...,I give consent,Hindu,
171,Male,44,Gurgaon,Have this life,Lonely,Was married earlier,No,Over 90 Lakhs,Entrepreneur,MiT pune,Masters,Not now,I give consent,Hindu,
172,Male,34,Gurugram,To cultivate so much self awareness to be your...,My growing up years were a bit challenging but...,Never married,No,Between 35-40 Lakhs,"Environment, Health & Safety Specialist","Faculty of Engineering , DEI",Masters,I am a high level of empath with kindness and ...,I give consent,Spiritual,
173,Female,36,Gurugram,Able to live life on my own terms,A mixed bag pf good times and bad. Military ki...,Never married,No,Between 50-70 Lakhs,Corporate and commercial lawyer,"University of York, UK",Masters,Respect and empathy are 2 core values that are...,I give consent,Hindu,


In [4]:
dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,5])

In [5]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
170,Male,29,Gurgaon,Never married
171,Male,44,Gurgaon,Was married earlier
172,Male,34,Gurugram,Never married
173,Female,36,Gurugram,Never married


In [6]:
age_weight =2.0
relationship_weight = 1.5
city_weight = 1.0


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

In [8]:
# one hot encoding
encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

In [9]:
encoded_categorical_features

,Never married,"Separated, filed for a divorce","Separated, not legally divorced",Was married earlier,Widow/ Widower
2,0,1,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
5,0,0,0,1,0
6,1,0,0,0,0
...,...,...,...,...,...
170,1,0,0,0,0
171,0,0,0,1,0
172,1,0,0,0,0
173,1,0,0,0,0


In [10]:
#label encode gender

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

In [11]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
170,Male,29,Gurgaon,Never married
171,Male,44,Gurgaon,Was married earlier
172,Male,34,Gurugram,Never married
173,Female,36,Gurugram,Never married


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

In [13]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
170,Male,29,Gurgaon,Never married
171,Male,44,Gurgaon,Was married earlier
172,Male,34,Gurugram,Never married
173,Female,36,Gurugram,Never married


In [14]:
labeledFeatures=labeledFeatures.reshape(-1,1)

print(city_vectorized.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
# prepare the data for training
features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized.toarray()])


In [16]:
print(features.shape)

(173, 50)


In [17]:
k =100
knn = NearestNeighbors(n_neighbors=k, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine', n_neighbors=100)

In [19]:
row =172
selectedRow = refDataset.iloc[row]

In [20]:
distances, indices = knn.kneighbors(features[row].reshape(1,-1))
print(indices)

[[172  34  98 164  87  47 162 115  93   5  31  16 154  58  74   3  15 120
   57  17  68 129 140   2  69  78  70 137  82  99  22 146  33 163   9  20
  152 125 148  80 160 147   7  90  64 158 157 168  37  72 116 102  89 150
  144 149  38 106   4  63 141 151 155 159  96 138  88  13  43   0 107 167
   60 103  44   6 145 127 134  18 124  95  40  49 153  85 117 131   8  59
   10  46 130 133 132 170  14  27  23  79]]


In [21]:
for i in indices[0]:
    if(dataset.iloc[i]['My Gender'] == selectedRow["My Gender"]):
        continue
    print(dataset.iloc[i].values)
    print("")
    print("")


['Male' 33 'Bangalore' 'Was married earlier']


['Male' 29 'Bangalore ' 'Never married']


['Male' 31 'Bangalore' 'Never married']


['Male' 31 'Bangalore ' 'Never married']


['Male' 32 'Bangalore' 'Never married']


['Male' 32 'Bangalore ' 'Never married']


['Male' 32 'Bangalore ' 'Never married']


['Male' 32 'Bangalore ' 'Never married']


['Male' 33 'Vadodara' 'Was married earlier']


['Male' 34 'Bangalore' 'Never married']


['Male' 34 'Bangalore' 'Never married']


['Male' 34 'Bangalore' 'Never married']


['Male' 35 'Bangalore ' 'Never married']


['Male' 35 'Hyderabad' 'Was married earlier']


['Male' 35 'Bangalore' 'Never married']


['Male' 35 'Pune' 'Was married earlier']


['Male' 35 'Chennai' 'Was married earlier']


['Male' 36 'Bangalore ' 'Never married']


['Male' 36 'Bangalore' 'Separated, filed for a divorce']


['Male' 36 'Bengaluru ' 'Was married earlier']


['Male' 36 'Bengaluru' 'Was married earlier']


['Male' 29 'Mumbai ' 'Never married']


['Male' 29 'Gurgaon

In [ ]:
def preprocessData(dataset):
    dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,5])

    vectorizer = TfidfVectorizer()
    city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

    # one hot encoding
    encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

    #label encode
    labelencoder = LabelEncoder()
    labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

    scaler = StandardScaler()
    scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

    labeledFeatures=labeledFeatures.reshape(-1,1)
    city_vectorized = city_vectorized.toarray()

    features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized])
    return features

In [ ]:
print(preprocessData(refDataset))

In [22]:
joblib.dump(knn, 'simpleclassifier.joblib')

['simpleclassifier.joblib']